#### NYC DOT Real-Time Traffic Speed Data

- [Kaggle](https://www.kaggle.com/datasets/aadimator/nyc-realtime-traffic-speed-data/data)
- [Web Archive](https://web.archive.org/web/20221006005747/https://data.cityofnewyork.us/Transportation/Real-Time-Traffic-Speed-Data/qkm5-nuaq)

In [2]:
import pandas as pd
from pandas.api.types import CategoricalDtype
import pyarrow as pa
import pyarrow.parquet as pq
from tqdm import tqdm

# --- 0. Конфигурация ---

csv_file = 'DOT_Traffic_Speeds_NBE.csv'
parquet_file = 'DOT_Traffic_Speeds_NBE.parquet'
chunksize = 1_000_000
n_lines = 65_631_862
columns_to_drop = ['STATUS', 'TRANSCOM_ID']
categorical_cols = ['BOROUGH', 'OWNER']
dtype_map = {
    'ID': 'int32',
    'LINK_ID': 'int32',
    'SPEED': 'float32',
    'TRAVEL_TIME': 'float32',
}

In [5]:
tmp = pd.read_csv(csv_file, chunksize=100).__next__().head()
tmp

,ID,SPEED,TRAVEL_TIME,STATUS,DATA_AS_OF,LINK_ID,LINK_POINTS,ENCODED_POLY_LINE,ENCODED_POLY_LINE_LVLS,OWNER,TRANSCOM_ID,BOROUGH,LINK_NAME
0,262,34.80,359,0,06/02/2017 11:41:59 PM,4616319,"40.6332305,-74.016151 40.63391,-74.01613 40.63...",ud_wF|gwbMgCCwATcBr@_BvAqDhGmGtL{AxB}AlAsBt@uB...,BBBBBBBBBBBBBBBBBBBBBBBBBBBB,NYC_DOT_LIC,4616319,Brooklyn,GOW S 9TH STREET - 7TH AVENUE
1,204,55.92,155,0,06/02/2017 11:41:59 PM,4616320,"40.7894406,-73.786291 40.78918,-73.78792 40....",_u}wFhkjaMr@dI~A~HtA|EbEnKxBdHv@~Cv@jGRhCJ~G[p...,BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB,NYC_DOT_LIC,4616320,Queens,CIP N TNB - Whitestone Expwy S Exit 14 (Linden...
2,106,39.77,159,0,06/02/2017 11:41:59 PM,4616323,"40.77158,-73.994441 40.7713004,-73.99455 40.77...",kezwFf`sbMv@TxAVnDZe@Gz@J~@Xf@VlEnC??~KpH??vCp...,BBBBBBBBBBBBBBBBB,NYC_DOT_LIC,4616323,Manhattan,12th Ave S 57th St - 45th St
3,184,65.24,39,0,06/03/2017 04:46:59 AM,4616253,"40.8347204,-73.86593 40.83357,-73.86199 40.832...",_pfxF`}yaMdFsWfDmPpH}^lEgTBBBBB,BBBBB,NYC_DOT_LIC,4616253,Bronx,CBE E TAYLOR AVENUE - CASTLE HILL AVENUE
4,3,14.91,422,0,06/02/2017 11:41:59 PM,4616324,"40.76375,-73.999191 40.763521,-73.99935 40.762...",mtxwF|}sbMl@^~GpK|LrIbLlH??lK~G|FtD`C~@}@WdWnG...,BBBBBBBBBBBBBBB,NYC_DOT_LIC,4616324,Manhattan,12th ave @ 45th - 11 ave ganservoort st


In [6]:
tmp['LINK_POINTS']

0    40.6332305,-74.016151 40.63391,-74.01613 40.63...
1    40.7894406,-73.786291  40.78918,-73.78792  40....
2    40.77158,-73.994441 40.7713004,-73.99455 40.77...
3    40.8347204,-73.86593 40.83357,-73.86199 40.832...
4    40.76375,-73.999191 40.763521,-73.99935 40.762...
Name: LINK_POINTS, dtype: object

In [16]:
# --- 1. Первый проход: Сбор всех уникальных категорий ---

print("--- Начало первого прохода: Сбор уникальных категорий ---")
unique_values = {col: set() for col in categorical_cols}

# Создаем итератор только для нужных столбцов, чтобы ускорить процесс
pass1_iterator = pd.read_csv(csv_file, usecols=categorical_cols, chunksize=chunksize)

for chunk in tqdm(pass1_iterator, total=n_lines // chunksize + 1):
    for col in categorical_cols:
        unique_values[col].update(chunk[col].dropna().unique())

print("Сбор уникальных значений завершен.")
for col, values in unique_values.items():
    print(f"  Найдено {len(values)} уникальных значений в столбце '{col}'.")

# Создаем общие типы данных для категорий
category_dtypes = {
    col: CategoricalDtype(categories=sorted(list(values)), ordered=False)
    for col, values in unique_values.items()
}

--- Начало первого прохода: Сбор уникальных категорий ---


 64%|██████▍   | 65/101 [02:42<01:29,  2.50s/it]

Сбор уникальных значений завершен.
  Найдено 6 уникальных значений в столбце 'BOROUGH'.
  Найдено 8 уникальных значений в столбце 'OWNER'.


In [17]:
unique_values

{'BOROUGH': {'Bronx',
  'Brooklyn',
  'Manhattan',
  'Queens',
  'Staten Island',
  'Staten island'},
 'OWNER': {'MTA Bridges & Tunnels',
  'NYC Thruway Tarrytown',
  'NYC-DOT-Region 10',
  'NYC_DOT_LIC',
  'PA - LINCOLN TUNNEL',
  'PA -Lincoln Tunnel',
  'PA-GWBridge',
  'Verrazano-Narrows-Bridge'}}

In [23]:
pd.read_csv(
    csv_file,
    iterator=True,
    chunksize=chunksize,
    # parse_dates=['DATA_AS_OF'],
    low_memory=True
).__next__().head()

,ID,SPEED,TRAVEL_TIME,STATUS,DATA_AS_OF,LINK_ID,LINK_POINTS,ENCODED_POLY_LINE,ENCODED_POLY_LINE_LVLS,OWNER,TRANSCOM_ID,BOROUGH,LINK_NAME
0,262,34.80,359,0,06/02/2017 11:41:59 PM,4616319,"40.6332305,-74.016151 40.63391,-74.01613 40.63...",ud_wF|gwbMgCCwATcBr@_BvAqDhGmGtL{AxB}AlAsBt@uB...,BBBBBBBBBBBBBBBBBBBBBBBBBBBB,NYC_DOT_LIC,4616319,Brooklyn,GOW S 9TH STREET - 7TH AVENUE
1,204,55.92,155,0,06/02/2017 11:41:59 PM,4616320,"40.7894406,-73.786291 40.78918,-73.78792 40....",_u}wFhkjaMr@dI~A~HtA|EbEnKxBdHv@~Cv@jGRhCJ~G[p...,BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB,NYC_DOT_LIC,4616320,Queens,CIP N TNB - Whitestone Expwy S Exit 14 (Linden...
2,106,39.77,159,0,06/02/2017 11:41:59 PM,4616323,"40.77158,-73.994441 40.7713004,-73.99455 40.77...",kezwFf`sbMv@TxAVnDZe@Gz@J~@Xf@VlEnC??~KpH??vCp...,BBBBBBBBBBBBBBBBB,NYC_DOT_LIC,4616323,Manhattan,12th Ave S 57th St - 45th St
3,184,65.24,39,0,06/03/2017 04:46:59 AM,4616253,"40.8347204,-73.86593 40.83357,-73.86199 40.832...",_pfxF`}yaMdFsWfDmPpH}^lEgTBBBBB,BBBBB,NYC_DOT_LIC,4616253,Bronx,CBE E TAYLOR AVENUE - CASTLE HILL AVENUE
4,3,14.91,422,0,06/02/2017 11:41:59 PM,4616324,"40.76375,-73.999191 40.763521,-73.99935 40.762...",mtxwF|}sbMl@^~GpK|LrIbLlH??lK~G|FtD`C~@}@WdWnG...,BBBBBBBBBBBBBBB,NYC_DOT_LIC,4616324,Manhattan,12th ave @ 45th - 11 ave ganservoort st


In [21]:
# --- 2. Второй проход: Конвертация и запись с помощью ParquetWriter ---

print("\n--- Начало второго прохода: Конвертация и запись данных ---")

# Определяем функцию для обработки одного чанка, чтобы избежать дублирования кода
def process_chunk(chunk, dtypes_map, cat_dtypes):
    chunk.drop(columns=columns_to_drop, inplace=True, errors='ignore')
    for col, dtype in dtypes_map.items():
        if col in chunk.columns:
            chunk[col] = chunk[col].astype(dtype, errors='ignore')
    for col, dtype in cat_dtypes.items():
        if col in chunk.columns:
            chunk[col] = chunk[col].astype(dtype, errors='ignore')
    return chunk

# Создаем итератор для второго прохода
pass2_iterator = pd.read_csv(
    csv_file,
    iterator=True,
    chunksize=chunksize,
    parse_dates=['DATA_AS_OF'],
    low_memory=True
)

# Обработка первого чанка для получения схемы
first_chunk = next(pass2_iterator)
first_chunk = process_chunk(first_chunk, dtype_map, category_dtypes)

# Конвертируем первый чанк в таблицу PyArrow и получаем её схему
schema = pa.Table.from_pandas(first_chunk, preserve_index=False).schema

# NEW: Создаем ParquetWriter с использованием with-statement для автоматического закрытия
with pq.ParquetWriter(parquet_file, schema, compression='snappy') as writer:
    print("Обработана и записана часть 1...")
    # Записываем первую таблицу
    writer.write_table(pa.Table.from_pandas(first_chunk, schema=schema, preserve_index=False))

    # Обрабатываем и записываем оставшиеся чанки
    for i, chunk in enumerate(pass2_iterator, start=2):
        processed_chunk = process_chunk(chunk, dtype_map, category_dtypes)
        
        # Конвертируем в таблицу и записываем
        table = pa.Table.from_pandas(processed_chunk, schema=schema, preserve_index=False)
        writer.write_table(table)
        
        print(f"Обработана и записана часть {i}...")

print(f"\nКонвертация завершена! Финальный файл '{parquet_file}' готов.")



--- Начало второго прохода: Конвертация и запись данных ---


/var/tmp/ipykernel_1645518/1262639778.py:26: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  first_chunk = next(pass2_iterator)


Обработана и записана часть 1...


/var/tmp/ipykernel_1645518/1262639778.py:39: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  for i, chunk in enumerate(pass2_iterator, start=2):


Обработана и записана часть 2...
Обработана и записана часть 3...


...

Обработана и записана часть 62...


/var/tmp/ipykernel_1645518/1262639778.py:39: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  for i, chunk in enumerate(pass2_iterator, start=2):


Обработана и записана часть 63...
Обработана и записана часть 64...


/var/tmp/ipykernel_1645518/1262639778.py:39: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  for i, chunk in enumerate(pass2_iterator, start=2):


Обработана и записана часть 65...

Конвертация завершена! Финальный файл 'DOT_Traffic_Speeds_NBE.parquet' готов.


In [24]:
import polars as pl

# Загрузка происходит почти мгновенно
df = pl.read_parquet('DOT_Traffic_Speeds_NBE.parquet')

# Можно сразу посмотреть информацию о типах данных
print(df)

# Пример анализа, который выполнится очень быстро
# Найдем среднюю скорость в час пик (8-9 утра) по районам
(
    df.filter(pl.col("DATA_AS_OF").dt.hour().is_between(8, 9))
    .group_by("BOROUGH")
    .agg(pl.col("SPEED").mean().alias("avg_morning_rush_speed"))
    .sort("avg_morning_rush_speed")
    # .collect() # Если бы мы использовали lazy scan
)


shape: (64_914_523, 11)
┌─────┬───────────┬────────────┬────────────┬───┬────────────┬────────────┬───────────┬────────────┐
│ ID  ┆ SPEED     ┆ TRAVEL_TIM ┆ DATA_AS_OF ┆ … ┆ ENCODED_PO ┆ OWNER      ┆ BOROUGH   ┆ LINK_NAME  │
│ --- ┆ ---       ┆ E          ┆ ---        ┆   ┆ LY_LINE_LV ┆ ---        ┆ ---       ┆ ---        │
│ i32 ┆ f32       ┆ ---        ┆ datetime[n ┆   ┆ LS         ┆ cat        ┆ cat       ┆ str        │
│     ┆           ┆ f32        ┆ s]         ┆   ┆ ---        ┆            ┆           ┆            │
│     ┆           ┆            ┆            ┆   ┆ str        ┆            ┆           ┆            │
╞═════╪═══════════╪════════════╪════════════╪═══╪════════════╪════════════╪═══════════╪════════════╡
│ 262 ┆ 34.799999 ┆ 359.0      ┆ 2017-06-02 ┆ … ┆ BBBBBBBBBB ┆ NYC_DOT_LI ┆ Brooklyn  ┆ GOW S 9TH  │
│     ┆           ┆            ┆ 23:41:59   ┆   ┆ BBBBBBBBBB ┆ C          ┆           ┆ STREET -   │
│     ┆           ┆            ┆            ┆   ┆ BBBBBBBB   ┆     

BOROUGH,avg_morning_rush_speed
cat,f32
"""Manhattan""",22.897594
"""Bronx""",32.115406
"""Brooklyn""",32.699806
"""Staten Island""",37.99342
"""Queens""",38.0
"""Staten island""",38.325562


In [25]:
df.head()

ID,SPEED,TRAVEL_TIME,DATA_AS_OF,LINK_ID,LINK_POINTS,ENCODED_POLY_LINE,ENCODED_POLY_LINE_LVLS,OWNER,BOROUGH,LINK_NAME
i32,f32,f32,datetime[ns],i32,str,str,str,cat,cat,str
262,34.799999,359.0,2017-06-02 23:41:59,4616319,"""40.6332305,-74.016151 40.63391…","""ud_wF|gwbMgCCwATcBr@_BvAqDhGmG…","""BBBBBBBBBBBBBBBBBBBBBBBBBBBB""","""NYC_DOT_LIC""","""Brooklyn""","""GOW S 9TH STREET - 7TH AVENUE"""
204,55.919998,155.0,2017-06-02 23:41:59,4616320,"""40.7894406,-73.786291 40.7891…","""_u}wFhkjaMr@dI~A~HtA|EbEnKxBdH…","""BBBBBBBBBBBBBBBBBBBBBBBBBBBBBB…","""NYC_DOT_LIC""","""Queens""","""CIP N TNB - Whitestone Expwy S…"
106,39.77,159.0,2017-06-02 23:41:59,4616323,"""40.77158,-73.994441 40.7713004…","""kezwFf`sbMv@TxAVnDZe@Gz@J~@Xf@…","""BBBBBBBBBBBBBBBBB""","""NYC_DOT_LIC""","""Manhattan""","""12th Ave S 57th St - 45th St"""
184,65.239998,39.0,2017-06-03 04:46:59,4616253,"""40.8347204,-73.86593 40.83357,…","""_pfxF`}yaMdFsWfDmPpH}^lEgTBBBB…","""BBBBB""","""NYC_DOT_LIC""","""Bronx""","""CBE E TAYLOR AVENUE - CASTLE H…"
3,14.91,422.0,2017-06-02 23:41:59,4616324,"""40.76375,-73.999191 40.763521,…","""mtxwF|}sbMl@^~GpK|LrIbLlH??lK~…","""BBBBBBBBBBBBBBB""","""NYC_DOT_LIC""","""Manhattan""","""12th ave @ 45th - 11 ave ganse…"
